<img src="logo_UNSAM.jpg" align="right" width="120">


# <center>Trabajo final: Análisis y procesamiento de señales: *Detección de crisis epilépticas mediante análisis de EEG*</center> 

## - Alumna: Lola Pampin

## - Profesor: Mariano Llamedo Soria

### <center>Febrero 2026</center> 

## <center> Universidad Nacional de San Martín, Escuela de Ciencia y Tecnología</center> 


